In [1]:
# table.views-table
#   tr
#     td.views-field-title
#     td.views-field-field-bias-image
#       a href="media-bias/left-center"

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

In [3]:
#url = 'https://www.allsides.com/media-bias/ratings'

In [4]:
url = 'https://www.allsides.com/media-bias/ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B%5D=2&field_news_source_type_tid%5B%5D=3&field_news_source_type_tid%5B%5D=4&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title='

In [5]:
bias_rating = {
    'left': -1,
    'left-center': -0.5,
    'center': 0,
    'allsides': 0,
    'right-center': 0.5,
    'right': 1,
}

In [6]:
def get_page(page=0):
    full_url = f'{url}&page={page}'
    res = requests.get(full_url)
    if not res.ok:
        raise f'Failed to open: {res.status_code}: {res.text}'
    return res

In [7]:
def get_rows(text):
    try:
        soup = BeautifulSoup(text, 'html.parser')
        table = soup.find('table', 'views-table')
        return table.find_all('tr')
    except AttributeError:
        return []

In [8]:
def parse_table_row(row):
    title = row.find('td', 'views-field-title').get_text().strip()
    bias = row.find('td', 'views-field-field-bias-image').find('a')['href']
    bias = bias.replace('/media-bias/', '')
    return (title, bias_rating[bias])

In [9]:
def read_page(page=0):
    res = get_page(page)
    rows = get_rows(res.text)
    return [parse_table_row(row) for row in rows[1:]]

In [10]:
page = 0
with open('media-bias.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    
    data = read_page(page)
    while len(data) > 0:
        print(f'Got {len(data)} rows for page {page}')
        for row in data:
            writer.writerow(row)
        page += 1
        data = read_page(page)

Got 50 rows for page 0
Got 50 rows for page 1
Got 50 rows for page 2
Got 50 rows for page 3
Got 50 rows for page 4
Got 50 rows for page 5
Got 50 rows for page 6
Got 50 rows for page 7
Got 50 rows for page 8
Got 50 rows for page 9
Got 50 rows for page 10
Got 46 rows for page 11
